# Feature Engineering

Some questions:
- West part vs Eest part playing style differences 3pt?
- What the most important key factors of increase playmin?
- With the increase the age, how ability change?
- how to predict playmin?

Methods:

boolean

## 01 Missing Values

Dataset source: https://www.kaggle.com/pablote/nba-enhanced-stats

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('2012-18_playerBoxScore.csv', encoding='utf_8')

In [3]:
def nas_sorted(df):
    return df.isnull().sum().sort_values(ascending = False)

In [4]:
df.shape

(155713, 45)

In [5]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,155463,155464,155465,155466,155467,155468,155469,155470,155471,155472,155473,155474,155475,155476,155477,155478,155479,155480,155481,155482,155483,155484,155485,155486,155487,155488,155489,155490,155491,155492,155493,155494,155495,155496,155497,155498,155499,155500,155501,155502,155503,155504,155505,155506,155507,155508,155509,155510,155511,155512,155513,155514,155515,155516,155517,155518,155519,155520,155521,155522,155523,155524,155525,155526,155527,155528,155529,155530,155531,155532,155533,155534,155535,155536,155537,155538,155539,155540,155541,155542,155543,155544,155545,155546,155547,155548,155549,155550,155551,155552,155553,155554,155555,155556,155557,155558,155559,155560,155561,155562,155563,155564,155565,155566,155567,155568,155569,155570,155571,155572,155573,155574,155575,155576,155577,155578,155579,155580,155581,155582,155583,155584,155585,155586,155587,155588,155589,155590,155591,155592,155593,155594,155595,155596,155597,155598,155599,155600,155601,155602,155603,155604,155605,155606,155607,155608,155609,155610,155611,155612,155613,155614,155615,155616,155617,155618,155619,155620,155621,155622,155623,155624,155625,155626,155627,155628,155629,155630,155631,155632,155633,155634,155635,155636,155637,155638,155639,155640,155641,155642,155643,155644,155645,155646,155647,155648,155649,155650,155651,155652,155653,155654,155655,155656,155657,155658,155659,155660,155661,155662,155663,155664,155665,155666,155667,155668,155669,155670,155671,155672,155673,155674,155675,155676,155677,155678,155679,155680,155681,155682,155683,155684,155685,155686,155687,155688,155689,155690,155691,155692,155693,155694,155695,155696,155697,155698,155699,155700,155701,155702,155703,155704,155705,155706,155707,155708,155709,155710,155711,155712
gmDate,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/30/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/12,10/31/1

In [6]:
sorted_nas = nas_sorted(df)

In [7]:
sorted_nas[:5]

opptDayOff    0
playSTL       0
playAST       0
playPTS       0
playBDate     0
dtype: int64

No missing value were found in this dataset

### Find outliers

In [8]:
df.columns = df.columns.str.lstrip()

In [9]:
# np.arange(-3, 4, 1)

In [10]:
from scipy import stats

def percentiles(column):
    z_scores = stats.zscore(column)
    # segment based on number of standard deviations away from the mean     
    hist, bin_edges = np.histogram(z_scores, bins=np.arange(-3, 4, 1), density=True)
    return np.stack((hist, bin_edges[1:]))

In [11]:
percentiles(df['opptDayOff'])

array([[ 0.        ,  0.03460392,  0.27512656,  0.65518408,  0.02965865,
         0.00542679],
       [-2.        , -1.        ,  0.        ,  1.        ,  2.        ,
         3.        ]])

In [12]:
def too_many_outliers(column, threshold = .05):
    #  expected .021 if normal distribution
    z_less_neg_two = percentiles(column)[0, 0]
    z_gt_two = percentiles(column)[0, -1]
    if z_less_neg_two > threshold or z_gt_two > threshold:
        return np.hstack((column.name, z_less_neg_two, z_gt_two))
    else:
        False

In [13]:
def outlier_columns(df, threshold = .05):
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    outlier_columns = np.array([too_many_outliers(df[column]) for column in numeric_columns])
    return np.array([column for column in outlier_columns if column is not None])

In [14]:
def select_outliers(column, upper_tail = True):
    if upper_tail:
        return column[stats.zscore(column) > 2]
    else:
        return column[stats.zscore(column) < -2]

In [15]:
np.seterr(divide='ignore', invalid='ignore') # to solve RuntimeWarning: invalid value encountered in divide
outlier_columns(df)

array([['playPF', '0.0', '0.05271236184518955'],
       ['play3P%', '0.0', '0.059288562933088436']], dtype='<U32')

In [16]:
select_outliers(df['playPF']).value_counts()

5    6695
6    1513
Name: playPF, dtype: int64

In [17]:
df['playPF'].value_counts(normalize=True)

1    0.247012
2    0.236422
0    0.192039
3    0.172600
4    0.099215
5    0.042996
6    0.009717
Name: playPF, dtype: float64

In [1]:
df['play3P%'].hist(bins=100)

NameError: name 'df' is not defined

'playPF' means Personal fouls made by player and the distribution looks reasonable

'play3P%' means Three point percentage made by player and it is also reasonable since a lot of players do not shoot 3

## 02 DateTimes

In [20]:
import pandas as pd
import numpy as np
df = pd.read_csv('2012-18_playerBoxScore.csv', encoding='utf_8')

In [21]:
pd.set_option('display.max_columns', 500)

In [22]:
df.shape

(155713, 45)

In [23]:
df.columns

Index(['gmDate', 'gmTime', 'seasTyp', 'playLNm', 'playFNm', 'teamAbbr',
       'teamConf', 'teamDiv', 'teamLoc', 'teamRslt', 'teamDayOff',
       'playDispNm', 'playStat', 'playMin', 'playPos', 'playHeight',
       'playWeight', 'playBDate', 'playPTS', 'playAST', 'playTO', 'playSTL',
       'playBLK', 'playPF', 'playFGA', 'playFGM', 'playFG%', 'play2PA',
       'play2PM', 'play2P%', 'play3PA', 'play3PM', 'play3P%', 'playFTA',
       'playFTM', 'playFT%', 'playORB', 'playDRB', 'playTRB', 'opptAbbr',
       'opptConf', 'opptDiv', 'opptLoc', 'opptRslt', 'opptDayOff'],
      dtype='object')

In [24]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
teamDayOff,155713.0,1.941238,1.072307,0.0,1.00,2.0000,2.0000,11.0
playMin,155713.0,22.924258,10.678114,0.0,15.00,24.0000,31.0000,60.0
playHeight,155713.0,79.012574,3.430492,69.0,77.00,79.0000,82.0000,89.0
playWeight,155713.0,217.970272,25.939346,149.0,200.00,220.0000,237.0000,355.0
playPTS,155713.0,9.695048,7.866322,0.0,4.00,8.0000,14.0000,70.0
playAST,155713.0,2.121917,2.474538,0.0,0.00,1.0000,3.0000,25.0
playTO,155713.0,1.304772,1.391012,0.0,0.00,1.0000,2.0000,12.0
playSTL,155713.0,0.734672,0.982164,0.0,0.00,0.0000,1.0000,10.0
playBLK,155713.0,0.460899,0.854966,0.0,0.00,0.0000,1.0000,12.0
playPF,155713.0,1.907792,1.455529,0.0,1.00,2.0000,3.0000,6.0


In [25]:
df.dtypes

gmDate         object
gmTime         object
seasTyp        object
playLNm        object
playFNm        object
teamAbbr       object
teamConf       object
teamDiv        object
teamLoc        object
teamRslt       object
teamDayOff      int64
playDispNm     object
playStat       object
playMin         int64
playPos        object
playHeight      int64
playWeight      int64
playBDate      object
playPTS         int64
playAST         int64
playTO          int64
playSTL         int64
playBLK         int64
playPF          int64
playFGA         int64
playFGM         int64
playFG%       float64
play2PA         int64
play2PM         int64
play2P%       float64
play3PA         int64
play3PM         int64
play3P%       float64
playFTA         int64
playFTM         int64
playFT%       float64
playORB         int64
playDRB         int64
playTRB         int64
opptAbbr       object
opptConf       object
opptDiv        object
opptLoc        object
opptRslt       object
opptDayOff      int64
dtype: obj

In [26]:
def find_object_features(df):
    return list(df.dtypes[df.dtypes == 'object'].index)

In [27]:
def find_object_feature_values(df):
    object_features = find_object_features(df)
    return df[object_features][:2].values

In [28]:
find_object_feature_values(df)[0, :3]

array(['10/30/12', '19:00', 'Regular'], dtype=object)

In [29]:
def contains_date(column):
#     remove nas first, potentially use all
    regex_string = (r'^\d{1,2}-\d{1,2}-\d{4}$|^\d{4}-\d{1,2}-\d{1,2}$' + 
'|^\d{1,2}\/\d{1,2}\/\d{4}$|^\d{4}\/\d{1,2}\/\d{1,2}$|^\d{1,2}\/\d{1,2}\/\d{1,2}$')
    return column.str.contains(regex_string).any()

In [30]:
def find_date_features(df):
    series_contains_date = df.apply(contains_date)
    return series_contains_date.index[series_contains_date.values]

In [31]:
date_features = find_date_features(df)

In [32]:
date_features

Index(['gmDate', 'playBDate'], dtype='object')

In [33]:
contains_date(df['gmDate'])

True

In [34]:
def to_dates(df):
    date_features = find_date_features(df)
    return df[date_features].astype('datetime64[ns]')

In [35]:
df_date_features = to_dates(df)

In [36]:
df_date_features.dtypes

gmDate       datetime64[ns]
playBDate    datetime64[ns]
dtype: object

In [37]:
df_date_features[:1]

,gmDate,playBDate
0,2012-10-30,1986-10-07


In [38]:
from date_lib import add_datepart

In [39]:
# copied_df = df.copy()
# add_datepart(copied_df, 'gmDate')

In [40]:
def generate_new_date_columns(dates_df):
    copied_dates_df = dates_df.copy()
    for col in copied_dates_df.columns:
        add_datepart(copied_dates_df, col)
    return copied_dates_df

In [41]:
new_date_col_df = generate_new_date_columns(df_date_features)

In [42]:
len(new_date_col_df.columns)

26

Once coercing our dates, we can merge with the original dataframe, and replace each of the old columns that should be dates.

In [43]:
def merge_dfs(original_df, new_df):
    copied_original = original_df.copy()
    date_features = find_date_features(original_df)
    copied_dropped = copied_original.drop(columns = date_features)
    copied_dropped[new_df.columns] = new_df
    return copied_dropped

In [44]:
new_df = merge_dfs(df, new_date_col_df)

We can confirm that the new_df has fewer object_feature columns than the original.

In [45]:
# new_df = new_df.drop(columns = remaining_date_features)
len(find_object_features(new_df))

17

In [46]:
len(find_object_features(df))

19

In [47]:
print(len(df.columns),len(new_df.columns))

45 69


In [48]:
new_df

,gmTime,seasTyp,playLNm,playFNm,teamAbbr,teamConf,teamDiv,teamLoc,teamRslt,teamDayOff,playDispNm,playStat,playMin,playPos,playHeight,playWeight,playPTS,playAST,playTO,playSTL,playBLK,playPF,playFGA,playFGM,playFG%,play2PA,play2PM,play2P%,play3PA,play3PM,play3P%,playFTA,playFTM,playFT%,playORB,playDRB,playTRB,opptAbbr,opptConf,opptDiv,opptLoc,opptRslt,opptDayOff,gmYear,gmMonth,gmWeek,gmDay,gmDayofweek,gmDayofyear,gmIs_month_end,gmIs_month_start,gmIs_quarter_end,gmIs_quarter_start,gmIs_year_end,gmIs_year_start,gmElapsed,playBYear,playBMonth,playBWeek,playBDay,playBDayofweek,playBDayofyear,playBIs_month_end,playBIs_month_start,playBIs_quarter_end,playBIs_quarter_start,playBIs_year_end,playBIs_year_start,playBElapsed
0,19:00,Regular,Price,A.J.,WAS,East,Southeast,Away,Loss,0,A.J. Price,Starter,29,PG,74,195,7,6,1,0,0,1,13,2,0.1538,4,0,0.0000,9,2,0.2222,1,1,1.0000,1,1,2,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1986,10,41,7,1,280,False,False,False,False,False,False,529027200
1,19:00,Regular,Ariza,Trevor,WAS,East,Southeast,Away,Loss,0,Trevor Ariza,Starter,25,SG,80,200,9,4,0,3,2,0,8,3,0.3750,4,1,0.2500,4,2,0.5000,2,1,0.5000,1,2,3,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1985,6,26,30,6,181,True,False,True,False,False,False,488937600
2,19:00,Regular,Okafor,Emeka,WAS,East,Southeast,Away,Loss,0,Emeka Okafor,Starter,25,C,82,255,10,0,1,0,4,1,10,4,0.4000,10,4,0.4000,0,0,0.0000,4,2,0.5000,5,2,7,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1982,9,39,28,1,271,False,False,False,False,False,False,402019200
3,19:00,Regular,Beal,Bradley,WAS,East,Southeast,Away,Loss,0,Bradley Beal,Starter,22,SG,77,202,8,3,2,1,0,1,8,2,0.2500,4,0,0.0000,4,2,0.5000,2,2,1.0000,0,3,3,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1993,6,26,28,0,179,False,False,False,False,False,False,741225600
4,19:00,Regular,Booker,Trevor,WAS,East,Southeast,Away,Loss,0,Trevor Booker,Starter,17,PF,79,240,4,1,4,1,1,4,9,2,0.2222,8,2,0.2500,1,0,0.0000,0,0,0.0000,1,0,1,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1987,11,48,25,2,329,False,False,False,False,False,False,564796800
5,19:00,Regular,Crawford,Jordan,WAS,East,Southeast,Away,Loss,0,Jordan Crawford,Bench,29,SG,76,195,11,5,1,1,1,1,13,4,0.3077,7,4,0.5714,6,0,0.0000,4,3,0.7500,1,2,3,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1988,10,42,23,6,297,False,False,False,False,False,False,593568000
6,19:00,Regular,Webster,Martell,WAS,East,Southeast,Away,Loss,0,Martell Webster,Bench,23,SG,81,210,9,1,0,2,0,1,6,4,0.6667,3,3,1.0000,3,1,0.3333,0,0,0.0000,0,3,3,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1986,12,49,4,3,338,False,False,False,False,False,False,534038400
7,19:00,Regular,Vesely,Jan,WAS,East,Southeast,Away,Loss,0,Jan Vesely,Bench,21,SF,83,235,7,1,1,0,1,3,4,3,0.7500,4,3,0.7500,0,0,0.0000,4,1,0.2500,3,1,4,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1990,4,17,24,1,114,False,False,False,False,False,False,640915200
8,19:00,Regular,Singleton,Chris,WAS,East,Southeast,Away,Loss,0,Chris Singleton,Bench,17,SF,80,225,4,2,1,2,0,2,7,2,0.2857,6,2,0.3333,1,0,0.0000,0,0,0.0000,2,2,4,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1989,11,47,21,1,325,False,False,False,False,False,False,627609600
9,19:00,Regular,Barron,Earl,WAS,East,Southeast,Away,Loss,0,Earl Barron,Bench,16,PF,84,250,8,0,0,1,1,2,6,4,0.6667,6,4,0.6667,0,0,0.0000,1,0,0.0000,4,4,8,CLE,East,Central,Home,Win,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1981,8,33,14,4,226,False,False,False,False,False,False,366595200


In [49]:
# new_df.drop(columns=['gmIs_month_end', 'gmIs_month_start','gmIs_quarter_end','gmIs_quarter_start','gmIs_year_end','gmIs_year_start','playBIs_month_end','playBIs_month_start','playBIs_quarter_end','playBIs_quarter_start','playBIs_year_end','playBIs_year_start'])

In [50]:
# new_df.to_csv(r'Adjusted_2012-18_playerBoxScore.csv')

## 03 Categorical Features

### Feature engineering

In [51]:
def find_object_features(new_df):
    return list(new_df.dtypes[new_df.dtypes == 'object'].index)

In [52]:
def find_object_feature_values(new_df):
    object_features = find_object_features(new_df)
    return new_df[object_features][:1].values[0]

In [53]:
def informative(new_df):
    non_informative = [column for column in new_df.columns if len(new_df[column].unique()) == 1]
    informative_columns = list(set(new_df.columns.to_list()) - set(non_informative))
    return new_df[informative_columns]

In [54]:
def percentage_unique(df_series):
    series_filled = df_series.dropna()
    return len(series_filled.unique())/len(series_filled)

In [55]:
def find_categorical(new_df, threshold = .5):    
    categorical_df = pd.DataFrame({})
    for column in new_df.columns:
        if percentage_unique(new_df[column]) < threshold:
            categorical_df[column] = new_df[column]
    return categorical_df 

In [56]:
df_informative = informative(new_df)

potential_categorical = find_categorical(df_informative)
# potential_categorical

### Combine with Selecting Categorical Columns

In [57]:
def summarize_counts(new_df):
    non_empty_columns = new_df.dropna(axis=1,how='all').columns
    frequencies = np.array([new_df[column].value_counts(normalize=True).values[0] for column in non_empty_columns]).reshape(-1, 1)
    columns = non_empty_columns.to_numpy().reshape(-1, 1)
    top_values = np.array([new_df[column].value_counts(normalize=True).index[0] for column in non_empty_columns]).reshape(-1, 1)
    summarize = np.hstack((columns, frequencies, top_values))
    return summarize[summarize[:,1].argsort()[::-1]]

In [58]:
summary = summarize_counts(potential_categorical)

In [59]:
summary

array([['playBIs_year_end', 0.9991779748640126, 'False'],
       ['gmIs_year_start', 0.9964036400300553, 'False'],
       ['playBIs_year_start', 0.9960183157475612, 'False'],
       ['gmIs_year_end', 0.9944192199752108, 'False'],
       ['playBIs_quarter_end', 0.9934430651262258, 'False'],
       ['gmIs_quarter_start', 0.9896668871577838, 'False'],
       ['gmIs_quarter_end', 0.9889733034492946, 'False'],
       ['playBIs_quarter_start', 0.983675094565001, 'False'],
       ['playBIs_month_end', 0.9812475515852883, 'False'],
       ['gmIs_month_end', 0.9672089035597542, 'False'],
       ['playBIs_month_start', 0.9642547507272996, 'False'],
       ['gmIs_month_start', 0.9625657459557005, 'False'],
       ['playBLK', 0.6937956368447079, '0'],
       ['play3P%', 0.5770231130348783, '0.0'],
       ['play3PM', 0.5770231130348783, '0'],
       ['playSTL', 0.5302511672114724, '0'],
       ['playStat', 0.526115353246036, 'Bench'],
       ['opptDayOff', 0.5168611483948032, '2'],
       ['teamDay

In [60]:
def selected_summaries(new_df, not_values = [], lower_bound = .1, upper_bound = 1):
    potential_cols = summarize_counts(new_df)
    potential_cols = potential_cols[potential_cols[:, 1] > lower_bound]
    potential_cols = potential_cols[potential_cols[:, 1] < upper_bound]
    not_tf = ~np.isin(potential_cols[:, 2], not_values)
    return potential_cols[not_tf]

In [61]:
selected = selected_summaries(new_df, not_values = ['TRUE', 'FALSE'], upper_bound = .90)
selected

array([['playBLK', 0.6937956368447079, '0'],
       ['play3P%', 0.5770231130348783, '0.0'],
       ['play3PM', 0.5770231130348783, '0'],
       ['playSTL', 0.5302511672114724, '0'],
       ['playStat', 0.526115353246036, 'Bench'],
       ['opptDayOff', 0.5168611483948032, '2'],
       ['teamDayOff', 0.5164501358268095, '2'],
       ['teamConf', 0.5029830521536416, 'West'],
       ['opptConf', 0.5013646901671666, 'West'],
       ['teamRslt', 0.5008445023857995, 'Win'],
       ['opptRslt', 0.5008445023857995, 'Loss'],
       ['teamLoc', 0.5004334898178059, 'Home'],
       ['opptLoc', 0.5004334898178059, 'Away'],
       ['playORB', 0.4945187620815218, '0'],
       ['playFT%', 0.4642001631206129, '0.0'],
       ['playFTM', 0.4642001631206129, '0'],
       ['playFTA', 0.4288017057021572, '0'],
       ['play3PA', 0.3501377534309916, '0'],
       ['playTO', 0.34967536429199875, '0'],
       ['playAST', 0.29815108565116594, '0'],
       ['playPF', 0.24701213129282718, '1'],
       ['playPos', 

In [62]:
def num_is_digit(array, str_index = 0):
    return np.array([value[str_index].isdigit() for value in array])

In [63]:
num_is_digit(selected[:, 2], str_index = 0)[0:10]

array([ True,  True,  True,  True, False,  True,  True, False, False,
       False])

In [64]:
def remove_digits_from_selected(selected_matrix, col_idx, str_indices = [0, -1]):
    for idx in str_indices:
        selected_col = selected_matrix[~num_is_digit(selected_matrix[:, col_idx], idx)]
    return selected_col

In [65]:
selected_sums_no_digits = remove_digits_from_selected(selected, 2, [0, -1])
selected_sums_no_digits

array([['playStat', 0.526115353246036, 'Bench'],
       ['teamConf', 0.5029830521536416, 'West'],
       ['opptConf', 0.5013646901671666, 'West'],
       ['teamRslt', 0.5008445023857995, 'Win'],
       ['opptRslt', 0.5008445023857995, 'Loss'],
       ['teamLoc', 0.5004334898178059, 'Home'],
       ['opptLoc', 0.5004334898178059, 'Away'],
       ['playPos', 0.2170274800434132, 'SG'],
       ['teamDiv', 0.16928580144239724, 'Southwest'],
       ['opptDiv', 0.16777661466929544, 'Pacific']], dtype=object)

### Cleaning Values

1. Find columns to clean

In [66]:
def categorical_plus_values(new_df, threshold = 5):
    categorical_cols = find_categorical(new_df)
    return [column for column in categorical_cols if len(new_df[column].value_counts()) > threshold]

In [67]:
selected_cat_cols = selected_sums_no_digits[:, 0]

selected_cat_cols

array(['playStat', 'teamConf', 'opptConf', 'teamRslt', 'opptRslt',
       'teamLoc', 'opptLoc', 'playPos', 'teamDiv', 'opptDiv'],
      dtype=object)

In [68]:
cat_cols_df = df_informative[selected_cat_cols]
cat_cols_df[:3]

,playStat,teamConf,opptConf,teamRslt,opptRslt,teamLoc,opptLoc,playPos,teamDiv,opptDiv
0,Starter,East,East,Loss,Win,Away,Home,PG,Southeast,Central
1,Starter,East,East,Loss,Win,Away,Home,SG,Southeast,Central
2,Starter,East,East,Loss,Win,Away,Home,C,Southeast,Central


In [69]:
updated_non_digits = categorical_plus_values(cat_cols_df)

In [70]:
len(updated_non_digits)

3

In [71]:
updated_non_digits

['playPos', 'teamDiv', 'opptDiv']

In [72]:
new_df[updated_non_digits].describe()

,playPos,teamDiv,opptDiv
count,155713,155713,155713
unique,7,6,6
top,SG,Southwest,Pacific
freq,33794,26360,26125


### Clean Values of Relevant Columns

In [73]:
def selected_cat_values(column, threshold = .02):
    values_counted = column.value_counts(normalize=True)
    return values_counted[values_counted > threshold]

In [74]:
selected = selected_cat_values(new_df.opptDiv, .02)

In [75]:
selected

Pacific      0.167777
Northwest    0.166993
Southwest    0.166595
Southeast    0.166306
Atlantic     0.166280
Central      0.166049
Name: opptDiv, dtype: float64

In [76]:
def reduce_cat_values(column, threshold = .02):
    column = column.copy()
    selected_values = selected_cat_values(column, threshold).index
    column[~column.isin(selected_values)] = 'other'
    column.astype('category')
    return column

In [77]:
new_neigh_cleansed =  reduce_cat_values(new_df.opptDiv, .02)

In [78]:
new_neigh_cleansed.value_counts(normalize = True)

Pacific      0.167777
Northwest    0.166993
Southwest    0.166595
Southeast    0.166306
Atlantic     0.166280
Central      0.166049
Name: opptDiv, dtype: float64

In [79]:
len(new_df[updated_non_digits].columns)

3

In [80]:
categoricals = ['playStat', 'teamConf', 'opptConf', 'teamRslt', 'opptRslt', 'opptLoc', 'teamLoc', 'playPos', 'teamDiv','opptDiv']

In [81]:
def df_reduced_categories(df, categoricals, threshold = .01):
    new_df = pd.DataFrame()
    for category in categoricals:
        new_df[category] = reduce_cat_values(df[category], threshold)
    return new_df

In [82]:
df_reduced = df_reduced_categories(df, categoricals)

In [83]:
df_reduced.describe()

,playStat,teamConf,opptConf,teamRslt,opptRslt,opptLoc,teamLoc,playPos,teamDiv,opptDiv
count,155713,155713,155713,155713,155713,155713,155713,155713,155713,155713
unique,2,2,2,2,2,2,2,6,6,6
top,Bench,West,West,Win,Loss,Away,Home,SG,Southwest,Pacific
freq,81923,78321,78069,77988,77988,77924,77924,33794,26360,26125


In [84]:
summarize_counts(df_reduced)

array([['playStat', 0.526115353246036, 'Bench'],
       ['teamConf', 0.5029830521536416, 'West'],
       ['opptConf', 0.5013646901671666, 'West'],
       ['opptRslt', 0.5008445023857995, 'Loss'],
       ['teamRslt', 0.5008445023857995, 'Win'],
       ['teamLoc', 0.5004334898178059, 'Home'],
       ['opptLoc', 0.5004334898178059, 'Away'],
       ['playPos', 0.2170274800434132, 'SG'],
       ['teamDiv', 0.16928580144239724, 'Southwest'],
       ['opptDiv', 0.16777661466929544, 'Pacific']], dtype=object)

In [85]:
def replace_df_columns(original_df, replacing_df):
    replacing_cols = replacing_df.columns
    original_df = original_df.drop(columns = replacing_cols)
    new_df = pd.concat([original_df, replacing_df], axis = 1)
    return new_df

In [86]:
updated_df = replace_df_columns(new_df, df_reduced)

In [87]:
updated_df

,gmTime,seasTyp,playLNm,playFNm,teamAbbr,teamDayOff,playDispNm,playMin,playHeight,playWeight,playPTS,playAST,playTO,playSTL,playBLK,playPF,playFGA,playFGM,playFG%,play2PA,play2PM,play2P%,play3PA,play3PM,play3P%,playFTA,playFTM,playFT%,playORB,playDRB,playTRB,opptAbbr,opptDayOff,gmYear,gmMonth,gmWeek,gmDay,gmDayofweek,gmDayofyear,gmIs_month_end,gmIs_month_start,gmIs_quarter_end,gmIs_quarter_start,gmIs_year_end,gmIs_year_start,gmElapsed,playBYear,playBMonth,playBWeek,playBDay,playBDayofweek,playBDayofyear,playBIs_month_end,playBIs_month_start,playBIs_quarter_end,playBIs_quarter_start,playBIs_year_end,playBIs_year_start,playBElapsed,playStat,teamConf,opptConf,teamRslt,opptRslt,opptLoc,teamLoc,playPos,teamDiv,opptDiv
0,19:00,Regular,Price,A.J.,WAS,0,A.J. Price,29,74,195,7,6,1,0,0,1,13,2,0.1538,4,0,0.0000,9,2,0.2222,1,1,1.0000,1,1,2,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1986,10,41,7,1,280,False,False,False,False,False,False,529027200,Starter,East,East,Loss,Win,Home,Away,PG,Southeast,Central
1,19:00,Regular,Ariza,Trevor,WAS,0,Trevor Ariza,25,80,200,9,4,0,3,2,0,8,3,0.3750,4,1,0.2500,4,2,0.5000,2,1,0.5000,1,2,3,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1985,6,26,30,6,181,True,False,True,False,False,False,488937600,Starter,East,East,Loss,Win,Home,Away,SG,Southeast,Central
2,19:00,Regular,Okafor,Emeka,WAS,0,Emeka Okafor,25,82,255,10,0,1,0,4,1,10,4,0.4000,10,4,0.4000,0,0,0.0000,4,2,0.5000,5,2,7,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1982,9,39,28,1,271,False,False,False,False,False,False,402019200,Starter,East,East,Loss,Win,Home,Away,C,Southeast,Central
3,19:00,Regular,Beal,Bradley,WAS,0,Bradley Beal,22,77,202,8,3,2,1,0,1,8,2,0.2500,4,0,0.0000,4,2,0.5000,2,2,1.0000,0,3,3,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1993,6,26,28,0,179,False,False,False,False,False,False,741225600,Starter,East,East,Loss,Win,Home,Away,SG,Southeast,Central
4,19:00,Regular,Booker,Trevor,WAS,0,Trevor Booker,17,79,240,4,1,4,1,1,4,9,2,0.2222,8,2,0.2500,1,0,0.0000,0,0,0.0000,1,0,1,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1987,11,48,25,2,329,False,False,False,False,False,False,564796800,Starter,East,East,Loss,Win,Home,Away,PF,Southeast,Central
5,19:00,Regular,Crawford,Jordan,WAS,0,Jordan Crawford,29,76,195,11,5,1,1,1,1,13,4,0.3077,7,4,0.5714,6,0,0.0000,4,3,0.7500,1,2,3,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1988,10,42,23,6,297,False,False,False,False,False,False,593568000,Bench,East,East,Loss,Win,Home,Away,SG,Southeast,Central
6,19:00,Regular,Webster,Martell,WAS,0,Martell Webster,23,81,210,9,1,0,2,0,1,6,4,0.6667,3,3,1.0000,3,1,0.3333,0,0,0.0000,0,3,3,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1986,12,49,4,3,338,False,False,False,False,False,False,534038400,Bench,East,East,Loss,Win,Home,Away,SG,Southeast,Central
7,19:00,Regular,Vesely,Jan,WAS,0,Jan Vesely,21,83,235,7,1,1,0,1,3,4,3,0.7500,4,3,0.7500,0,0,0.0000,4,1,0.2500,3,1,4,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1990,4,17,24,1,114,False,False,False,False,False,False,640915200,Bench,East,East,Loss,Win,Home,Away,SF,Southeast,Central
8,19:00,Regular,Singleton,Chris,WAS,0,Chris Singleton,17,80,225,4,2,1,2,0,2,7,2,0.2857,6,2,0.3333,1,0,0.0000,0,0,0.0000,2,2,4,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1989,11,47,21,1,325,False,False,False,False,False,False,627609600,Bench,East,East,Loss,Win,Home,Away,SF,Southeast,Central
9,19:00,Regular,Barron,Earl,WAS,0,Earl Barron,16,84,250,8,0,0,1,1,2,6,4,0.6667,6,4,0.6667,0,0,0.0000,1,0,0.0000,4,4,8,CLE,0,2012,10,44,30,1,304,False,False,False,False,False,False,1351555200,1981,8,33,14,4,226,False,False,False,False,False,False,366595200,Bench,East,East,Loss,Win,Home,Away,PF,Southeast,Central


## 04 Removing a specified number of features

### Onto Recursive Feature Elimination

Write out the code for performing RFE:
our goal is to reduce the number of columns by one, by removing the column whose removal hurts the least

#### Using SKLearn

In [88]:
number_df = updated_df.loc[:, 'playHeight':'playTRB']

In [89]:
from sklearn.model_selection import train_test_split
X = number_df
y = updated_df['playMin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=30)

In [90]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

estimator = LinearRegression(n_jobs = -1)
rfe = RFE(estimator, 6, step = 1)

In [91]:
number_df.shape

(155713, 23)

In [92]:
selected_data = rfe.fit_transform(number_df, updated_df['playMin'])

In [93]:
type(selected_data)

numpy.ndarray

In [94]:
rfe.support_

array([False, False, False, False, False, False, False, False,  True,
       False, False,  True, False, False,  True, False, False, False,
       False, False,  True,  True,  True])

In [95]:
import numpy as np
np.array(number_df.columns)[rfe.support_]

array(['playFGA', 'play2PA', 'play3PA', 'playORB', 'playDRB', 'playTRB'],
      dtype=object)

### Now that have data, perform linear regression

In [96]:
model = LinearRegression()
model.fit(selected_data, updated_df['playMin'])
model.score(selected_data, updated_df['playMin'])

0.6817655074547607

## 04 Number of Features

Exploring different levels

In [97]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import numpy as np

selectors = []

for idx in list(range(1, 8, 1))[::-1]:
    # 1. create an estimator
    estimator = LinearRegression(n_jobs=-1)
    # 2. perform rfe with the idx number of features
    rfe = RFE(estimator, idx, step=1)
    # 3. reduce the dataset to the number of features
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    
    # 4. compute the accuracy score and append to the list
    estimator.fit(X_train_rfe,y_train)
    score = estimator.score(X_test_rfe,y_test)
    selectors.append(np.array([rfe, score, idx]))

In [98]:
wrapped_selectors = np.array(selectors)

In [99]:
y_values = wrapped_selectors[:, 1]
x_values = wrapped_selectors[:, -1]

In [100]:
x_values

array([7, 6, 5, 4, 3, 2, 1], dtype=object)

In [101]:
import plotly

In [102]:
from graph import trace_values, plot
trace = trace_values(x_values, y_values)
plot([trace])

### Choosing the correct number

In [103]:
top_selector = selectors[-3]
top_selector

array([RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False),
  n_features_to_select=3, step=1, verbose=0),
       0.6195508034742333, 3], dtype=object)

In [104]:
top_rfe = top_selector[0]
np.array(number_df.columns)[top_rfe.support_]

array(['playFGA', 'play2PA', 'play3PA'], dtype=object)

### A question of efficiency

In [105]:
selectors = []

for idx in list(range(1, 8, 1))[::-1]:
    estimator = LinearRegression(n_jobs=-1)
    rfe = RFE(estimator, idx, step=1)
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)

    estimator.fit(X_train_rfe,y_train)
    score = estimator.score(X_test_rfe,y_test)
    selectors.append(np.array([rfe, score, idx]))

In [106]:
from sklearn.feature_selection import RFECV

Now RFECV stands for recursive feature engineering with cross validation. We'll explain cross validation below, but for now just recognize that the below code will also perform RFE at each level.

In [107]:
from sklearn.feature_selection import RFECV

estimator = LinearRegression(n_jobs=-1)
rfe = RFECV(estimator, step=1, cv=5)
X_train_rfe = rfe.fit_transform(X_train,y_train)

In [108]:
rfecv_scores = rfe.grid_scores_
rfecv_scores

array([0.37621443, 0.54236394, 0.54982472, 0.55977915, 0.63461704,
       0.63595176, 0.64719866, 0.65813321, 0.66565525, 0.67317029,
       0.68143306, 0.68695959, 0.69097859, 0.70827907, 0.7252811 ,
       0.73693284, 0.75204517, 0.75317037, 0.75420568, 0.75504258,
       0.75538152, 0.75565862, 0.7564369 ])

In [109]:
num_features = list(range(0,100))

In [110]:
from graph import plot, trace_values
trace_rfecv = trace_values(num_features, rfecv_scores)
plot([trace])